In [1]:
%load_ext blackcellmagic

In [2]:
import pandas as pd
import numpy as np
import openai
import json
import os
import datetime
import streamlit as st
from PIL import Image

In [3]:
# getting keys
# TODO: remove unnecessary aspects
with open('../config.json') as f:
    keys = json.load(f)
PATH = keys['path']
# openai_organization = keys['openai_organization']
# openai.organization = openai_organization
# openai_api_key = keys['openai_api_key']
# openai.api_key = openai_api_key
openai.api_key = keys['openai_api_key']

# Variables

In [4]:
teachers_dict = {
    'Severus Snape': 'Very sarcastic',
    'Aristoteles': 'wise, philosophical',
    'A professional programming teacher':'neutral',
    'Bob Ross':'very kind, understanding'
}

In [5]:
# Accessing a key
teacher = 'Severus Snape'
style = teachers_dict[teacher]

In [6]:
# add code input as text here
code_input = str('''
def save_to_txt(name, input_string, timestamp):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string) 
        '''
)

In [8]:
PATH = keys['path']
DATA_PATH = os.path.join(PATH, 'data/')

# Template

## Original

## step separation

# Template adoption

## Appending messages

In [8]:
# Accessing a key
teacher = 'Severus Snape'
style = teachers_dict[teacher]

# Define the code input
code_input = '''
def save_to_txt(name, input_string, timestamp):
    # Define the file name with the timestamp
    # TODO: add the PATH to the filename
    filename = f"../data/{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string) 
'''

# Create the messages list
messages = [
    { 'role': 'system', 'content': f'You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it.' },
    { 'role': 'user', 'content': f'Please review my following code: {code_input}' }
]


In [9]:
messages

[{'role': 'system',
  'content': 'You are Severus Snape and you teach programming learners. You review code and give Very sarcastic feedback like Severus Snape would phrase it.'},
 {'role': 'user',
  'content': 'Please review my following code: \ndef save_to_txt(name, input_string, timestamp):\n    # Define the file name with the timestamp\n    # TODO: add the PATH to the filename\n    filename = f"../data/{timestamp}_{name}.txt"\n    # Write the string to the file\n    with open(filename, "w") as file:\n        file.write(input_string) \n'}]

In [10]:
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages
    )

In [12]:
response_message = response["choices"][0]["message"]

In [13]:
response_message

<OpenAIObject at 0x7fd4011f1720> JSON: {
  "content": "Ah, a piece of code. How lovely. Let's have a closer look, shall we?\n\nFirst, we have a function named save_to_txt. Fascinating. However, I must admit, it lacks proper documentation. One might wonder what this function does exactly. But who needs clarity, right?\n\nNext, we have the parameter names. How imaginative. Name, input_string, and timestamp. I applaud your creativity. Truly inspiring.\n\nMoving on, we have a todo comment. Ah, the classic sign of an unfinished task. How impressive. Pray tell, when will you add the PATH to the filename? Or shall we leave it as a mystery, like your intentions behind this code?\n\nAnd now we arrive at the favorite part of any code review - the file name generation. Marvelous. Simply marvelous. Concatenating the timestamp, an underscore, and the name variable. I must say, your naming conventions are truly a sight to behold. An absolute masterpiece.\n\nFinally, we reach the pinnacle of this cod

In [14]:
def create_feedback(teacher, style, code_input):
    # defining the prompt templates for a standardized input
    template="You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template="{code_input}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # get a chat completion from the formatted messages
    chat_prompt.format_prompt(teacher=teacher, style=style, code_input=code_input).to_messages()

    llm = ChatOpenAI(temperature=0.9)
    chain = LLMChain(llm=llm, prompt=chat_prompt)

    feedback = (chain.run({
        'teacher':teacher, 
        'style':style, 
        'code_input':code_input
        }))

    return feedback

## Functions

**Experiment**
* unify get_latest_goal and evaluate_code into a functions functions

In [9]:
latest_goal = "handle errors when reading config file"

In [10]:
def learning_goal_append(latest_goal):
    global_goal = "\\n\\maintain code clarity & organization by separating code into sections/functions"
    goals = global_goal+','+latest_goal
    goal_dict = {
        "goals":goals
    }
    return json.dumps(goal_dict)

In [11]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [12]:
learning_goal_append(latest_goal="handle errors when reading config file")

'{"goals": "\\\\n\\\\maintain code clarity & organization by separating code into sections/functions,handle errors when reading config file"}'

In [13]:
get_current_weather(location='Boston', unit="fahrenheit")

'{"location": "Boston", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [14]:
# Step 1: send the conversation and available functions to GPT
messages = [
    {
        'role': 'system',
        'content': f'You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn.'
    },
    {
        'role':'user',
        'content':f'Please compare my code input: {code_input} with my learning goals. Did I improve my style and follow what I wanted to achieve?'
    }
]

In [15]:
functions = [
    {
        "name": "learning_goal_append",
        "description": "Extend the global learning goal with a new learning goal",
        "parameters": {
            "type": "object",
            "properties": {
                "latest_goal": {
                    "type": "string",
                    "description": "The latest goal which should be appended to the global goal",
                },
            },
            "required": ["latest_goal"],
        },
    }
]

In [16]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

In [17]:
response_message = response["choices"][0]["message"]

In [18]:
response_message

<OpenAIObject at 0x7fd588026ae0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"latest_goal\": \"Improve coding style and follow the learning goals\"\n}",
    "name": "learning_goal_append"
  },
  "role": "assistant"
}

In [20]:
# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
    
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "learning_goal_append": learning_goal_append,
    }  # only one function in this example, but you can have multiple
    
    function_name = response_message["function_call"]["name"]
    
    function_to_call = available_functions[function_name]
    
    function_args = json.loads(response_message["function_call"]["arguments"])
    
    function_response = function_to_call(
        latest_goal=function_args.get("latest_goal"),
    )

In [21]:
# Step 4: send the info on the function call and function response to GPT
messages.append(response_message)  # extend conversation with assistant's reply

In [22]:
messages

[{'role': 'system',
  'content': 'You are Severus Snape and you review code and give Very sarcastic feedback like Severus Snape would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn.'},
 {'role': 'user',
  'content': 'Please compare my code input: \ndef save_to_txt(name, input_string, timestamp):\n    # Define the file name with the timestamp\n    # TODO: add the PATH to the filename\n    filename = f"../data/{timestamp}_{name}.txt"\n    # Write the string to the file\n    with open(filename, "w") as file:\n        file.write(input_string) \n         with my learning goals. Did I improve my style and follow what I wanted to achieve?'},
 <OpenAIObject at 0x7fd588026ae0> JSON: {
   "content": null,
   "function_call": {
     "arguments": "{\n  \"latest_goal\": \"Improve coding style and follow the learning goals\"\n}",
     "name": "learning_goal_append"
   },
   "role": "assistant"
 }]

In [24]:
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)  # extend conversation with function response

In [26]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)  # get a new response from GPT where it can see the function response

In [27]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7YalpYliJi13qphhSlOaHNWeZsBme at 0x7fd5b3e8ce00> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Well, well, well, let's take a look at your code and see how well you've improved your coding style and followed your learning goals.\n\nFirst, let's talk about code clarity and organization. It seems that your code is divided into sections, which is a good start. However, I would highly recommend separating the functionality into separate functions to improve code maintainability and organization. It's always a good idea to have standalone functions responsible for specific tasks.\n\nNow, let's move on to coding style. Ah, the famous \"TODO\" comment. How delightful! It seems that you have intended to add the PATH to the filename but forgot to actually do it. Just a little reminder, my dear student, that it's important to complete all the TODOs in your code.\n\nAs for following yo

In [ ]:
# uses the learning goals retrieved in `get_latest_goal` if available and evaluates the code
def evaluate_code(teacher, style, code_input, latest_goal):
    if latest_goal is not None:
        # defining the prompt templates for a standardized input
        template = "You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn."
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        human_template = "Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        # get a chat completion from the formatted messages
        chat_prompt.format_prompt(
            teacher=teacher, style=style, code_input=code_input, latest_goal=latest_goal
        ).to_messages()

        llm = ChatOpenAI(temperature=0.9)
        chain = LLMChain(llm=llm, prompt=chat_prompt)

        evaluation = chain.run(
            {
                "teacher": teacher,
                "style": style,
                "code_input": code_input,
                "latest_goal": latest_goal,
            }
        )
    else:
        evaluation = "You haven't defined learning goals yet. If you want, you can define some by clicking the button below."
    return evaluation

In [6]:
print(run_conversation())

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The weather in Boston is currently 72 degrees Fahrenheit, sunny, and windy.",
        "role": "assistant"
      }
    }
  ],
  "created": 1688315282,
  "id": "chatcmpl-7XuDaDiMPzhdNWYC0xN4EotzytHsR",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 72,
    "total_tokens": 88
  }
}


# Educator

In [ ]:

# getting keys
# TODO: remove unnecessary aspects
with open('config.json') as f:
    keys = json.load(f)
PATH = keys['path']
DATA_PATH = os.path.join(PATH, 'data/')
openai_api_key = keys['openai_api_key']

os.environ["OPENAI_API_KEY"] = openai_api_key
directory = '/data' # replace this when working with PATH




# Variables 

# Get the current date and time
now = datetime.datetime.now()
# Format the date and time as a string in the desired format
timestamp = now.strftime("%Y%m%d%H%M%S")
# the available teachers and their characteristic style
teachers_dict = {
    'Severus Snape': 'Very sarcastic',
    'Aristoteles': 'wise, philosophical',
    'A professional programming teacher':'neutral',
    'Bob Ross':'very kind, understanding'
}



# Functions

def save_to_txt(name, input_string, timestamp, data_path):
    # Create the directory if it doesn't exist
    os.makedirs(data_path, exist_ok=True)
    # Define the file name with the timestamp
    filename = f"{data_path}{timestamp}_{name}.txt"
    # Write the string to the file
    with open(filename, "w") as file:
        file.write(input_string)

def create_feedback(teacher, style, code_input):
    # defining the prompt templates for a standardized input
    template="You are {teacher} and you teach programming learners. You review code and give {style} feedback like {teacher} would phrase it."
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    human_template="{code_input}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # get a chat completion from the formatted messages
    chat_prompt.format_prompt(teacher=teacher, style=style, code_input=code_input).to_messages()

    llm = ChatOpenAI(temperature=0.9)
    chain = LLMChain(llm=llm, prompt=chat_prompt)

    feedback = (chain.run({
        'teacher':teacher, 
        'style':style, 
        'code_input':code_input
        }))

    return feedback

def shorten_feedback(feedback): 
    # defining the prompt template for a standardized input
    short_feedback_prompt = PromptTemplate(
        input_variables=["feedback"],
        template="Please shorten the aspects of the following feeback: {feedback}?",
    )
    # initializing the LM
    # TO OPTIMIZE THE TEMPERATURE
    short_feedback_llm = OpenAI(temperature=0)
    # a simple chain taking the formatted the prompt and sending it to the LM
    short_feedback_chain = LLMChain(llm=short_feedback_llm, prompt=short_feedback_prompt)
    # Run the chain only specifying the input = short feedback.
    short_feedback = short_feedback_chain.run(feedback)
    return short_feedback

# retrieve learning goals from the saved files
def get_latest_goal(data_path):
    # Get a list of all files in the directory
    files = os.listdir(data_path)
    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_learninggoals.txt") and len(f) == 32]
    if not files:
        latest_goal = None
    else:
        # Get the most recent file
        files.sort(reverse=True)
        latest_file = files[0]
        # Read the contents of the file into the latest_goal variable
        with open(os.path.join(data_path, latest_file), "r") as f:
            latest_goal = f.read()
    return latest_goal

# uses the learning goals retrieved in `get_latest_goal` if available and evaluates the code
def evaluate_code(teacher, style, code_input, latest_goal):
    if latest_goal is not None:
        # defining the prompt templates for a standardized input
        template = "You are {teacher} and you review code and give {style} feedback like {teacher} would phrase it. You compare learning goals of your student with submitted code. If necessary, you remind your student what they wanted to learn."
        system_message_prompt = SystemMessagePromptTemplate.from_template(template)
        human_template = "Please compare my code input: {code_input} with my learning goals: {latest_goal}. Did I improve my style and follow what I wanted to achieve?"
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
        chat_prompt = ChatPromptTemplate.from_messages(
            [system_message_prompt, human_message_prompt]
        )

        # get a chat completion from the formatted messages
        chat_prompt.format_prompt(
            teacher=teacher, style=style, code_input=code_input, latest_goal=latest_goal
        ).to_messages()

        llm = ChatOpenAI(temperature=0.9)
        chain = LLMChain(llm=llm, prompt=chat_prompt)

        evaluation = chain.run(
            {
                "teacher": teacher,
                "style": style,
                "code_input": code_input,
                "latest_goal": latest_goal,
            }
        )
    else:
        evaluation = "You haven't defined learning goals yet. If you want, you can define some by clicking the button below."
    return evaluation

def pick_latest_shortfeedbacks(data_path):
    # TODO: use PATH here for directory
    # Get a list of all files in the directory
    files = os.listdir(data_path)
    # Filter the list to only include files with the correct format
    files = [f for f in files if f.endswith("_shortfeedback.txt") and len(f) == 32]
    # if there are fewer than three files take all of them, if there are more take latest 3
    if len(files)<3:
        # Get the three most recent files
        latest_files = files
    else:
        # Sort the list of files by date, with the most recent file first
        files.sort(reverse=True)
        # Get the three most recent files
        latest_files = files[:3]
    return latest_files

# takes a list of latest files and appends their content
def append_shortfeedbacks(latest_files):
    # Read the contents of the two files into string variables
    file_contents = []
    for file in latest_files:
        with open(os.path.join(DATA_PATH, file), "r") as f:
            file_contents.append(f.read())
    # Combine the three file contents into a single string variable
    latest_short_feedbacks = "\n".join(file_contents)
    return latest_short_feedbacks

def define_goal(latest_short_feedbacks):
    # defining the prompt template for a standardized input
    learning_goal_prompt = PromptTemplate(
        input_variables=["short_feedback"],
        template="Please summarize the following points: {short_feedback}",
    )
    # initializing the LM
    # TODO: check for optimal LLM
    learning_goal_llm = OpenAI(temperature=0.5)
    # a simple chain taking user input, formatting the prompt and sending it to the LM
    learning_goal_chain = LLMChain(llm=learning_goal_llm, prompt=learning_goal_prompt)
    # Run the chain only specifying the input variable.
    learning_goal = learning_goal_chain.run(latest_short_feedbacks)
    
    return learning_goal

def add_vertical_space(num_lines: int = 1):
    """Add vertical space to your Streamlit app."""
    for _ in range(num_lines):
        st.write("")

        

# streamlit page
st.set_page_config(
    page_title="The Educator",
    layout="wide",
)

def main():

    st.sidebar.image('background.png', use_column_width=True)

    
    st.title('Educator')

    # default teacher style
    teacher = 'Aristoteles'

    # two columns to design the page split
    col1, col2 = st.columns([3,1])

    with col1:
        st.write('')
    with col2:
        with st.expander("Choose your educator here"):
            # overwrite if wanted:
            teacher = st.radio(label='Who do you want to teach you?', options=[
                'Aristoteles', 'Severus Snape', 'A professional programming teacher', 'Bob Ross'
            ])

        # Accessing the style of the selected or default teacher via the dict
        style = teachers_dict[teacher]

    # two columns to keep the page split
    col3, col4 = st.columns([3,1])
    with col3:
        # Code review
        # add code input as text here
        code_input = st.text_area("Your code")

        # saving the input to txt via the prepared function
        save_to_txt(name='codeinput', input_string=code_input, timestamp=timestamp, data_path=DATA_PATH)

        # execute only when a code is passed 
        if code_input != '':
            # generate the feedback to the submitted code
            feedback = create_feedback(teacher, style, code_input)
            # saving the feedback to txt
            save_to_txt(name='feedback', input_string=feedback, timestamp=timestamp, data_path=DATA_PATH)
            # create shorter versions of the feedback for better further use
            short_feedback = shorten_feedback(feedback)
            # saving the short feedback to txt via the prepared function
            save_to_txt(name='shortfeedback', input_string=short_feedback, timestamp=timestamp, data_path=DATA_PATH)

            st.header('Your feedback:')
            st.write(feedback)

            # retrieving the current leaning goals for evaluation
            latest_goal = get_latest_goal(data_path=DATA_PATH)
            # evaluating the code against the current learning goals and 
            # prints out a reminder if no learning goals have been defined yet
            evaluation = evaluate_code(teacher, style, code_input, latest_goal)
            st.subheader('Think of your learning goals:')
            st.write(evaluation)


        add_vertical_space(2)


        # Set learning goals
        # This has to be done only if the user checks the button 
        if st.button('Define learning goals'):
            # selecting the last three feedbacks in their short form
            latest_files = pick_latest_shortfeedbacks(data_path=DATA_PATH)
            # combining the content of these files in one string
            latest_short_feedbacks = append_shortfeedbacks(latest_files=latest_files)
            # defining the new learning goals from the last feedbacks
            new_learning_goal = define_goal(latest_short_feedbacks=latest_short_feedbacks)
            # saving the input to txt via the prepared function
            save_to_txt(name='learninggoals', input_string=new_learning_goal, timestamp=timestamp, data_path=DATA_PATH)
            assert len(new_learning_goal) != 0, 'The created learning goal was empty'
            st.write('Your new goals are:' + new_learning_goal)

    with col4:
        image_name = str('teacher_images/') + teacher + str('.png')
        image = Image.open(image_name)
        st.image(image)


if __name__ == '__main__':
    main()
